In [1]:
import pandas as pd

In [2]:
df_raw = pd.read_excel('data/Teeltplan 2025.xlsx')

/Users/svenvanderburg/miniforge3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [3]:
df_raw.head()

,Gewas,Ras,Aantal vakken,Bed\nnummer,Z / PL / JONG,Combinatieteelt,Afstand tussen rij,Afstand in rij,Planten per vak,Planten nodig,...,Plantdatum,Verwachte oogst (week),Startdatum oogst,Eindweek\noogst,Einddatum oogst,Dagen tot volgroeiing,Dagen de tuin,Dagen in opkweek,Leverancier,Opmerkingen
0,Aardbei,?,2.0,NaN,NaN,NaN,15.0,15.0,4,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Prei (laat),Herstreuzen 2,1.0,NaN,PL,NaN,40.0,15.0,4,4.0,...,NaN,35.0,NaN,48.0,NaN,NaN,NaN,NaN,bolster,NaN
3,Knoflook,NaN,3.0,NaN,Z,NaN,NaN,NaN,9,27.0,...,NaN,28.0,NaN,28.0,NaN,NaN,NaN,NaN,NaN,NaN
4,Olifantenknoflook,NaN,3.0,NaN,PL,NaN,NaN,NaN,1,3.0,...,NaN,28.0,NaN,28.0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_raw.columns

Index(['Gewas', 'Ras', 'Aantal vakken', 'Bed\nnummer', 'Z / PL / JONG',
       'Combinatieteelt', 'Afstand tussen rij', 'Afstand in rij',
       'Planten per vak', 'Planten nodig', 'Zaaien (week)', 'Zaaidatum',
       'Planten (week)', 'Plantdatum', 'Verwachte oogst (week)',
       'Startdatum oogst', 'Eindweek\noogst', 'Einddatum oogst',
       'Dagen tot volgroeiing', 'Dagen de tuin', 'Dagen in opkweek',
       'Leverancier', 'Opmerkingen'],
      dtype='object')

In [5]:
def get_unique_bed_numbers(df):
    cleaned = df['Bed\nnummer'].str.replace(" ","").str.split(',').dropna()
    return {e for l in cleaned for e in l}

data = list()
mapper = {
    'zaaien': 'Zaaien (week)',
    'planten': 'Planten (week)'} 
for week in range(2, 53):
    for activity in mapper.keys():
        selection = df_raw[df_raw[mapper[activity]] == week]
        for gewas in set(selection['Gewas']):
            this_rows = selection[selection['Gewas'] == gewas]
            # bed_numbers = get_unique_bed_numbers(this_rows)
            # if bed_numbers and next(iter(bed_numbers)) == '?':
            #     print(f'skipping {gewas} becaue of no bednumbers')
            #     continue
            if len(this_rows['Z / PL / JONG'].unique()) > 2:
                raise ValueError(f'Multiple methods for {gewas}')
            method = this_rows['Z / PL / JONG'].unique()[0]
            if activity in ['zaaien', 'planten'] and method == 'DZ':
                to_do = 'direct zaaien'
            elif activity == 'zaaien' and method == 'TP':
                to_do = 'voorzaaien'
                bed_numbers = []
            else:
                to_do = activity

                
            data.append({'Week': week,
                         'Wat doen?': f'{gewas} {to_do}'})
in_time_df = pd.DataFrame(data)

In [6]:
in_time_df.head()

,Week,Wat doen?
0,3,Eikenbladsla (kas vroeg) zaaien
1,5,Spinazie (vroeg) zaaien
2,6,Chilipeper zaaien
3,7,Eikenbladsla (kas vroeg) planten
4,8,Ridderspoor zaaien


In [7]:
to_add = [
    {'Week': 13, 'Wat doen?': 'Roos, olijfboom snoeien'},
    {'Week': 13, 'Wat doen?': 'Vlinderstruik, braam snoeien'},
    {'Week': 10, 'Wat doen?': 'Rabarber scheuren'},
    {'Week': 14, 'Wat doen?': 'Lavendel snoeien'},
    {'Week': 6, 'Wat doen?': 'Winterjasmijn snoeien'}
]
in_time_df = pd.concat([in_time_df, pd.DataFrame(to_add)])
in_time_df = in_time_df.sort_values(by='Week')

In [8]:
in_time_df.to_excel('data/planning-per-week.xlsx', index=False)